Collecte de données(WebScraping)

In [ ]:
import requests  # Send HTTP requests to fetch web page content
from bs4 import BeautifulSoup  # Parse and extract data from HTML content
import tqdm.notebook as tqdm  # Display progress bars in Jupyter notebooks
import time  # Manage delays between requests
import pandas as pd  # Work with data in DataFrame format
import random  # Add delays between requests to avoid being blocked
import json  # Work with JSON data (save/load structured data)
import tqdm  # General tqdm for progress bars in scripts or terminal

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
}

listing_urls = set()

for i in tqdm.trange(500):  # trange = range loop with progress bar
    url = f'https://www.avito.ma/fr/maroc/voitures_d_occasion-%C3%A0_vendre?o={i+1}'
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Error {response.status_code} while accessing {url}")
        continue

    soup = BeautifulSoup(response.content, "html.parser")

    for a in soup.select('div.listing>a'):
        price_tag = a.select_one('p.dJAfqm.eTHoJR')
        if not price_tag or price_tag.get_text() == "Prix non spécifié":
            continue

        href = a.get('href')
        if href:
            listing_urls.add(href)

    time.sleep(random.uniform(1, 3))  # Pause to avoid getting blocked

print(f"Nombre total d'annonces collectées: {len(listing_urls)}")

  0%|          | 0/500 [00:00<?, ?it/s]

Nombre total d'annonces collectées: 11108


In [ ]:
# Initialize the dataset list
dataset = []

# Iterate over the listing URLs
for url in tqdm.tqdm(listing_urls):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Dictionary to store information for each car
    entry = {}

    # Extract price
    prix_elem = soup.select_one("p.lnEFFR")
    entry["Prix"] = prix_elem.get_text(strip=True) if prix_elem else None

    # Main attribute keys
    attribute_keys = [
        "Année", "BoiteàV", "Carburant", "kilometrage", "Marque", "Model", 
        "NBporte", "Origine", "Première main", "Puissance fiscale", "Etat"
    ]

    # Extract attribute values
    divs = soup.select('div.sc-19cngu6-1.doRGIC')
    attribute_values = set()  
    
    for i, attribute_key in enumerate(attribute_keys):
        if i < len(divs):
            value_elem = divs[i].select_one('span.fjZBup')
            value = value_elem.get_text(strip=True) if value_elem else None
            entry[attribute_key] = value
            if value:
                attribute_values.add(value) 
            entry[attribute_key] = None  # Reset the value (probably by mistake)

    # Get all spans containing features and attributes
    all_values = {
        span.get_text(strip=True)
        for span in soup.select('div.sc-19cngu6-1.doRGIC span.fjZBup')
    }

    # Remove values already used as attributes
    equipements = list(all_values - attribute_values)

    # Convert features to JSON format (clean list representation)
    entry['équipements'] = json.dumps(equipements)

    # Add entry to the dataset
    dataset.append(entry)


 79%|███████▊  | 8727/11108 [54:40<14:11,  2.80it/s]     

In [ ]:
# Save the data to a CSV file
df = pd.DataFrame(dataset)
df.to_csv('avito_pfm2.csv', index=False)
df.head()

NameError: name 'dataset' is not defined